In [229]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "2022-12-29",
	"end_date": "2022-12-31",
	"hourly": ["temperature_2m", "dew_point_2m", "rain", "snowfall", "surface_pressure", "cloud_cover", "et0_fao_evapotranspiration", "wind_speed_10m", "soil_temperature_0_to_7cm", "soil_moisture_0_to_7cm"],
	"timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_rain = hourly.Variables(2).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(3).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(4).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(5).ValuesAsNumpy()
hourly_et0_fao_evapotranspiration = hourly.Variables(6).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(7).ValuesAsNumpy()
hourly_soil_temperature_0_to_7cm = hourly.Variables(8).ValuesAsNumpy()
hourly_soil_moisture_0_to_7cm = hourly.Variables(9).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe['time'] = (hourly_dataframe['date'].dt.hour)/24
hourly_dataframe['month'] = (hourly_dataframe['date'].dt.month)/12
print(hourly_dataframe)
print(hourly_dataframe.dtypes)



Coordinates 52.5483283996582°N 13.407821655273438°E
Elevation 38.0 m asl
Timezone b'Europe/Berlin' b'CEST'
Timezone difference to GMT+0 7200 s
                        date  temperature_2m  dew_point_2m  rain  snowfall  \
0  2022-12-28 22:00:00+00:00        6.908500      4.558500   0.0       0.0   
1  2022-12-28 23:00:00+00:00        6.908500      4.308500   0.1       0.0   
2  2022-12-29 00:00:00+00:00        7.308500      5.408500   0.2       0.0   
3  2022-12-29 01:00:00+00:00        7.858500      5.908500   0.3       0.0   
4  2022-12-29 02:00:00+00:00        8.358500      5.908500   0.0       0.0   
..                       ...             ...           ...   ...       ...   
67 2022-12-31 17:00:00+00:00       16.108500     10.458500   0.0       0.0   
68 2022-12-31 18:00:00+00:00       16.258501     10.258500   0.0       0.0   
69 2022-12-31 19:00:00+00:00       15.708500      9.858500   0.0       0.0   
70 2022-12-31 20:00:00+00:00       15.358500      9.158501   0.0       0.0   

In [245]:
data = hourly_dataframe.select_dtypes(include=['float'])
meansloaded = pd.read_csv('dane\means6080.csv', skiprows=1, header=None)
stdsloaded = pd.read_csv('dane\stds6080.csv', skiprows=1, header=None)
npdata = data.to_numpy()
npmeans = meansloaded[1].to_numpy()
npstds = stdsloaded[1].to_numpy()

In [246]:
dane = (npdata - npmeans)/npstds


In [247]:
import tensorflow as tf


In [248]:
import numpy as np

In [249]:
model = tf.keras.models.load_model("model_6080.h5")

In [250]:
dane = dane.reshape(1,72,12)

In [251]:
preds = (model.predict(dane[-72:]) * stdsloaded[1].to_numpy() + meansloaded[1].to_numpy())


1/1 [==============================] - 1s 633ms/step


In [252]:
def generowanie_pogody(stdsloaded, meansloaded, iledni, pogoda):
    for _ in range(iledni*2):
        pogoda = np.concatenate((pogoda, pomoc(pogoda)), axis=1)
    return pogoda * stdsloaded[1].to_numpy() + meansloaded[1].to_numpy()

    
def pomoc(pogoda):
    return (model.predict(pogoda[:,-72:]))# * stdsloaded[1].to_numpy() + meansloaded[1].to_numpy())

In [256]:
pogoda = generowanie_pogody(stdsloaded, meansloaded, 365, dane)
pogoda = pogoda[:, 72:]


1/1 [==============================] - 0s 43ms/step


In [257]:
sequence_prediction = pogoda[0,:]
df = pd.DataFrame(sequence_prediction)
df.columns = stdsloaded[0]

df['month'] = (round(df['month']*12)).astype(int)
df['time'] = (round(df['time']*24)).astype(int)
df['time'] = df['time'].replace(-1, 0)

In [258]:
df.to_csv('dane\pogodaprzew6080_rok.csv', index=False)

In [244]:
df

,temperature_2m,dew_point_2m,rain,snowfall,surface_pressure,cloud_cover,et0_fao_evapotranspiration,wind_speed_10m,soil_temperature_0_to_7cm,soil_moisture_0_to_7cm,time,month
0,15.381316,8.585905,0.074059,0.007139,1005.400124,49.941180,0.073865,26.551699,11.410157,0.371997,22,12
1,14.776233,8.364075,0.044402,0.003701,1005.254263,52.331068,0.064135,26.092094,11.180030,0.371509,23,12
2,14.521121,8.340723,0.078802,0.001030,1005.224201,53.899919,0.059455,25.440878,11.090767,0.369742,1,12
3,14.131503,8.531049,0.090886,0.000916,1004.836562,57.905912,0.045554,25.632187,10.814172,0.367760,2,12
4,13.749128,8.850069,0.100836,0.000330,1004.758847,56.173403,0.038801,24.997462,10.715545,0.370294,2,12
...,...,...,...,...,...,...,...,...,...,...,...,...
235,-0.777457,-3.668141,-0.014146,0.013747,1022.021029,59.049846,0.025088,11.113978,1.027547,0.345531,17,5
236,-0.950615,-3.798618,-0.010086,0.014142,1022.114689,59.476417,0.023175,11.272918,1.047678,0.345671,18,5
237,-1.419365,-3.922491,-0.004889,0.012765,1022.376921,59.182399,0.026643,11.503365,1.022182,0.345779,19,5
238,-1.706857,-4.109569,0.008115,0.012028,1022.241480,59.619249,0.028647,11.277333,0.865576,0.345863,20,5
